# Introduction to SHAP values

This notebook aims to clarify the steps described in [Lundberg and Lee 2017](https://arxiv.org/abs/1705.07874), with examples and applications 

In [2]:
import numpy as np

def nonlinear_func(x):
    """This function defines a simple nonlinear function of 3 variables.
    The three variables correspond to the columns of the input matrix x."""
    return (0.7 + 1.3 * x[:, 0] -2.1 * x[:, 1]**2 + 1.5 * x[:, 2]**3)

print(nonlinear_func(np.array([[0.8, -0.1, 0.2]])))

[1.731]


We generate an input dataset of 10,000 triplets of values, where the values are generated according to a multivariate normal distribution with high correlation between the predictors.

In [2]:
np.random.seed(3141592)
means = [0, 0, 0]
covmat = [[1, 0.8, 0.9],
          [0.8, 1, 0.85],
          [0.9, 0.85, 1]]
x = np.random.multivariate_normal(mean=means, cov=covmat, size=10000)

In [ ]:
np.corrcoef(x.T)

## Additive Attribution Models

> The best explanation of a simple model is the model itself.

In the case of a *linear model* with **uncorrelated** predictors, the interpretation of the model is straightforward, since
$\mathbb{E}[f | {x_1, x_2, \ldots, x_p}] = \beta_0 + \beta_1 x_1 + \ldots \beta_p x_p$ and $\mathbb{E}[f | X_i = x_i + 1] - \mathbb{E}[f | X_i = x_i] = \beta_i$.

If the presence of multicollinearity the interpretation is less obvious since the value of a predictor depends on the value of the others. In the extreme case of two almost identical predictors, there is a whole subspace of predictor values that produce the same output, and intepretability at the *single* predictor level becomes impossible.

The fact that linear model with uncorrelated coefficient are the blueprint of an interpretable model suggests two things:

1. The interpretability of a model $f(x)$ can be quantified via another model $g(z)$.
2. $g(z)$ is a linear model.

**TODO** can we prove that the coefficients of $g(z)$ are independent?


### Explanation Methods

Let $f: x \in \mathcal{X} \subseteq \mathbb{R}^N \to \mathcal{Y}$ be the original model. It can be regression, classification, or something else.

**Definition**: an *explanation model* is any intepretable approximation of the original model. **Important**: We indicate with $x$, $z$ etc. the inputs of $f$,  the original model. Similarly, we indicate with $x'$, $z'$ etc. the inputs to $g$, the explanation model. These inputs are called **simplified inputs** because they are often a simplified form of the original input. In our discussion below we will *always* assume that the simplified input is a binary vector, *i.e.*, $z' \in \{0, 1\}^M$.

Since the explanation model approximates the original one, the output space of the explanatory model $g$ is the same as the output space of $f$, *i.e.*, $\mathcal{Y}$.

We assume that there is a function $h_x$ that maps the simplified input to the original one: $h_x(x'): x' \to x$ such that $h_x(x') = x$. This is not an analytical function, but rather a lookup table. The reason is that $x' \in {0, 1}^M$ contains (much) less information than $x$.

#### Local Explanation Methods

We assume that $g(z)$ is a **local explanation method**. This means the output of $f(x)$ is interpreted based solely on the input $x$, and not on any other $x \in \mathcal{X} \subseteq \mathbb{R}^N$.

This is of **crucial importance**: it means that every pair $\{f(x), y\}$ has *its own explanation model*. The *functional form* of the explanation model is the same for all $\{f(x), y\}$ pairs, but the coefficient of the model vary from pair to pair.


An **important assumption** in local methods is that, given two simplified vectors $x', z'$ such that $x' \approx z'$, and $f_x(x') = x$, then $g(z') \approx f(h_x(x')) = f(x)$.



**My Note**: I struggle with the idea of $x' \approx z'$. If these are two binary vectors, either they are identical, or they differ by a finite number of bits. Let's assume that $x'$ and $z'$ differ by one single bit. This often represents the presence/absence of a, possibly very important, feature. How is it then reasonable to expect that the $g(z') \approx f(x)$?

## Additive Feature Attribution Methods

Based on the above discussion, it will not be a surprise that we restrict our attention to local explanation models that have a linear functional form. More precisely

$$
\begin{equation}
g(z) = \phi_0 + \sum_{i=1}^M \phi_i z_i
\end{equation}
$$

where $z_i \in \{0, 1\}$ and $\phi \in \mathbb{R}$.

The equation above assigns an effect $\phi_i$ to each *simplified* feature $z_i$. **ARE THESE EFFECTS INDEPENDENT?**.


### LIME

LIME is presented as an example of a local linear explanation model. Simplified inputs are in the form described above, although their mapping to the original input depends on the data modality (text, images, etc). LIME uses the following optimization method to find the coefficients $\phi_i$:

$$
g(z) = \arg \min_{g \in \mathcal{G}} L(f, g, \pi_{x'}) + \Omega(g)
$$

Where $L$ is a squared loss, $\pi$ is a weighting kernel, and $\Omega(g)$ is a regularizing term. We said that each $\{x, f(x)\}$ pair has its own set of coefficients, however here the squared loss is minimized
> over a set of samples in the simplified input space weighted by the kernel $\pi_{x'}$.

#### My Understanding

We fix $x$ and have a value $f(x)$. $x'$ is the simplified input associated with $x$, and $h_x(x') = x$. If $z'$ is another simplified input "close" to $x'$, the weighting kernel will give it a non-negligible weight. Conversely, if $z'$ and $x'$ are "far apart" the kernel will set the contribution of $z'$ close to 0. Given $N$ inputs, the loss can be written as

$$
L(f, g, \pi_{x'}) = \sum_{i=1}^N \left[f(h_x(x')) - g(z'^{(i)})\right]^2 \pi_{x'}(z'^{(i)})
$$

Where $z^{(i)}$ is the i-th simplified input vector.

**VERIFY THIS PART**

### Classic Shapley Value Estimation

This approach was developed to deal with **linear models in presence of multicollinearity**. Let therefore be

$f(x) = \beta_0 + \beta_1 x_1 + \ldots \beta_p x_p$

Where $p$ is the total number of predictors. Let $F = \{x_1, \ldots, x_p\}$ be the set of all $p$ features and let $S \subset F$ be the subset of $F$ where we remove the i-th feature, i.e., $S = F \setminus \{i\}$. There are $2^{p-1}$ possible subsets, including the empty set $\emptyset$. We then compare the model on the subset $S$, and on the subset $S \cup \{i\}$, i.e., the same subset augmented by the feature we removed. We have:

$ \Delta f_{i} = f_{S \cup \{i\}}(x_{S \cup \{i\}}) - f_S(x_S)$

In a linear model $\Delta f_{i} = \beta_i^S$ where we must add the superscript $S$ to clarify that the value of $\beta_i$ will be different for every set $S$ we consider (we are fitting a different model every time).

## Simple Properties Uniquely Determine Additive Feature Attributions

If we require an additive feature attribution model to satisfy the three properties listed below, game theory guarantees that there is one and only one way to determine the coefficients $\phi_i$ as to satisfy these requirements.

### Local Accuracy

When $h_x(x') = x$, we require $g(x') = f(x)$, *i.e.*,
$g(x') = f(h_x(x')) = f(x)$

### Missingness

We assume, as usual, that $x' \in {0, 1}^M$. If the simplified input represent feature presence, *i.e.*,
$$
x_i' = \left\{
    \begin{array}{c l}
    1 &  \textrm{if }x_i \textrm{ is present} \\
    0 &  \textrm{otherwise}
    \end{array}
    \right.
$$
then $x_i' = 0 \implies \phi_i = 0$.

It is straightforward to remove a feature from a linear model. It is less obvious what this implies in a more complex model. We will see in the next section how to deal with this issue.

### Consistency

Let's assume that there are two models, $f$ and $f'$. For simplicity, let's indicate $f_x(z') = f(h_x(z'))$ and $f_x'(z') = f'(h_x(z'))$. Let's indicate with $z' \setminus i$ setting $z_i' = 0$.

If, for any two models $f$, $f'$
$$
f_x'(z') - f_x'(z' \setminus i) \geq f_x(z') - f_x(z' \setminus i)
$$
for all simplified inputs $z' \in \{0, 1\}^M$, then $\phi_i(f', x) \geq \phi_i(f, x)$.

In words, if for every two functions $f$ and $f'$, the effect of removing a feature from any of the simplified inputs from $f'$ is always larger or equal than the corresponding effect in $f$, then it has to be that $\phi_i(f', x) \geq \phi_i(f, x)$. Note that we write $\phi_i(f, x)$ to stress the fact that the value of $\phi_i$ depends on the function $f$ *and* on the specific input $x$.

## Theorem 1

Only one possible explanation model g follows Definition 1 and satisfies Properties 1, 2, and 3:
$$
\phi_i(f, x) = \sum_{z' \subseteq x'} \frac{|z'|!(M - |z'| - 1)!}{M!} [f_x(z') - f_x(z' \setminus i)]
$$
where $|z'|$ is the number of non-zero entries in $z'$, and $z' \subseteq x'$ represents all $z'$ vectors where the non-zero entries are a subset of the non-zero entries in $x'$.

Note that the theorem says nothing about the form of the mapping function $h_x(x')$. What it says is that for any such choice, there is only one way to define the $\phi(f, x)$ that satisfies additive feature attributions and the three properties above. Irrespective of the form of $h_x(x')$ we are still assuming that $x' \in \{0, 1\}^M$.

## SHAP (SHapley Additive exPlanation) Values

Let $z$ be an input of $f$. Let $z'$ be the corresponding simplified input. Let's define $h_x(z')$ in such a way that $f_x(z') = f(h_x(z')) = \mathbb{E}[f(z) | z_S]$ where $S$ is the set of non-zero indices in $z'$. An example will help clarify the situtation.

Let $z = \{0.2, -1.3, 2.1, 1.4\}$ an input vector and $z' = \{1, 1, 0, 1\}$ the corresponding simplified input. Therefore, assuming that we count indices starting from 1, it is $S = \{1, 2, 4\}$. This choice is sensible, since index 0 is reserved for the intercept. Then, based on the above definition, we have:
$$
f(h_x(z')) = \mathbb{E}[f(z) | z_S] \approx \frac{1}{N} \sum_{k=1}^N f(0.2, -1.3, x_3^{(k)}, 1.4)
$$
where we are summing on all input vectors $x^{(k)}$ for $k=1, 2, \ldots, N$.

According to the paper, ''implicit in this definition of SHAP values is a simplified input mapping $h_x(z') = z_S$''.

In order to compute a single $\phi(f, x)$ we need to consider all subsets $z' \subseteq x'$, *i.e.*, all subsets of non-zero indices in $x'$, and for each such subset, we need to compute an expectation value. This is clearly very expensive. For this reason, the authors describe two model-agnostic and four model-specific approximations. Feature independence and linearity can simplify computations as shown below

In general: $f(h_x(z')) = \mathbb{E}[f(z) | z_S] = \mathbb{E}_{z_{\bar S} | z_S}[f(z)]$. The latter simply means that we are computing the expectations over the indices that are *not* in $S$.

If the features are all independent, $\mathbb{E}_{z_{\bar S} | z_S}[f(z)] \approx \mathbb{E}_{z_{\bar S}}[f(z)]$. If the features are not independent, the value of a feature depends on the value of one or more other features. **CLARIFY THIS PART**

If the model is linear and the features are independent, the expression is just $\mathbb{E}_{z_{\bar S}}[f(z)] \approx f([z_S, \mathbb{E}[z_{\bar S}]])$.

### Theorem 2 (Shapley kernel)

LIME, more precisely, linear LIME, is an additive feature attribution model, but the way it estimates the coefficients $\phi_i$ is very different than the approach in Theorem 1. There is, however, a way to use the same approach and quadratic loss function, while still satisfying the  three properties. This requires redefining the kernel function and the regularization term.

Under the definition of the LIME loss, the specific forms of $\pi_{x'}$, $L$, and $\Omega$ that make the solutions consistent with the three prpoerties are:
$$
\begin{align*}
\Omega(g) &= 0 \\
\pi_{x'}(z') &= \frac{M - 1}{{M \choose |z'|} |z'| (M - |z'|)} \\
L(f, g, \pi_{x'}) &= \sum_{z' \in Z} [f(h_x^{-1}(z')) - g(z')]^2 \pi_{x'}(z')
\end{align*}
$$
where $|z'|$ is the number of non-zero elements in $z'$. Note that $\pi_{x'}(z') = \infty$ if $|z'| = 0$ or $|z'| = M$. This enforces that $\phi_0 = f_x(\emptyset)$ and $f(x) = \sum_{i=0}^M \phi_i$.

LIME uses simplified inputs that are equivalent to the assumption of a linear $f$ with independent coefficients, namely that $f(h_x(z')) = f([z_S, \mathbb[{Z_{\bar S}}]])$. With these assumptions, the $\phi_i$ can be simultaneously estimated via weighted linear regression.
**CLARIFY THIS POINT**

![shap values](./img/shap_values.jpg)

### Linear SHAP

Let $f(x) = \sum_{j=1}^M w_j x_j + b$ with independent features $x_j$. Then $\phi_0(f, x) = b$ and $\phi_i(f, x) = w_j ( x_j - \mathbb{E}[x_j])$. In other words, under the hypothesis above, the SHAP values can be estimated directly from the weights $w_j$, the features $x_j$, and their expectations.